In [14]:
! pip install --user tensorflow
! pip install --user numpy
! pip install --user tensorflow_datasets
! pip install --user matplotlib
! pip install --user scikit-image

     |████████████████████████████████| 14.0 MB 3.4 MB/s eta 0:00:01     |██████████████▋                 | 6.4 MB 15.3 MB/s eta 0:00:01
     |████████████████████████████████| 2.0 MB 10.6 MB/s eta 0:00:01
     |████████████████████████████████| 3.4 MB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 195 kB 4.7 MB/s eta 0:00:01
     |████████████████████████████████| 6.9 MB 8.1 MB/s eta 0:00:01     |██████████████████████          | 4.8 MB 490 kB/s eta 0:00:05
     |████████████████████████████████| 41.6 MB 1.7 MB/s eta 0:00:01     |██████████████████▉             | 24.5 MB 7.0 MB/s eta 0:00:03     |███████████████████████████▎    | 35.5 MB 3.9 MB/s eta 0:00:02     |███████████████████████████▋    | 35.9 MB 3.9 MB/s eta 0:00:02
ERROR: imageio 2.19.2 has requirement pillow>=8.3.2, but you'll have pillow 7.0.0 which is incompatible.


In [1]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.callbacks import TensorBoard
from keras.preprocessing.image import load_img, array_to_img, img_to_array,ImageDataGenerator 
from skimage import io, color
from keras.models import Sequential
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose

2022-05-22 17:57:18.820581: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-22 17:57:18.820616: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/curlyz/.local/lib/python3.8/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [2]:
print(tf.__version__)


2.8.0


In [3]:
def preprocessImg(path):
    # image = keras.preprocessing.image.img_to_array(keras.preprocessing.image.load_img(path))
    # image = np.array(image, dtype=float)
    image= io.imread(path)
    if image.shape[0] < 360 or image.shape[1] < 360:
        return
    image = image[0:360,0:360]
    rgb = image[:,:,0:3]
    # print(rgb.shape[0])
    lab = color.rgb2lab(rgb)
    # print(lab)
    #L channel is the input to network
    L = lab[:,:,0]
    #A and B channel is the output from network
    AB = lab[:,:,1:]
    AB /= 128
    # print(AB.shape[0], AB.shape[1])
    
    #reshape L abd AB tensors
    #this means a batch of 1 image of shape 400x400
    L = L.reshape(360, 360, 1)
    AB = AB.reshape(360, 360, 2)
    return L, AB

In [33]:
Ltrain = []
ABtrain = []
for filename in os.listdir('input/'):
    if os. path. isdir('input/'+filename):
        continue
    try:
        L, AB = preprocessImg('input/'+filename)
        Ltrain.append(L)
        ABtrain.append(AB)
    except:
        continue


In [31]:
def model(Ltrain, ABtrain):
    # Building the neural network
    model = Sequential()
    model.add(InputLayer(input_shape=(None, None, 1)))
    model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
    model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
    model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
    model.compile(optimizer='rmsprop',loss='mse')
    
    
#     datagen = ImageDataGenerator()
#         # shear_range=0.2,
#         # zoom_range=0.2,
#         # rotation_range=20,
#         # horizontal_flip=True)

#     # Generate training data
#     batch_size = 10
#     def imageGen(batch_size):
#         for Lbatch, ABbatch in datagen.flow(Ltrain, ABtrain, batch_size=batch_size):
#             yield (Lbatch.reshape(Lbatch.shape+(1,)), ABbatch)

    # Train model      
    # tensorboard = TensorBoard(log_dir="output/first_run")
    # model.fit(imageGen(batch_size), epochs=1, steps_per_epoch=10)
    model.fit(Ltrain, ABtrain, batch_size = 10, epochs=10)
    # model_json = model.to_json()
    # with open("model.json", "w") as json_file:
    #     json_file.write(model_json)
    # model.save_weights("model.h5")

#     model.fit(x=L, 
#     y=AB,
#     batch_size=1,
#     epochs=1000)
    model.save('./tmp/model')
    return model

In [37]:
def convertImg(img, model):
    L, AB = preprocessImg(img)
    L = L.reshape(1, 360, 360, 1)
    # print(L.shape)
    output = model.predict(L)
    # print(output.shape)
    output *= 128
    cur = np.zeros((360, 360, 3))
    cur[:,:,0] = L[0][:,:,0]
    cur[:,:,1:] = output[0]
    return cur
def saveImg(img, name):
    colored = "output/" + name + ".png"
    gray = "output/" + name + "_gray.png"
    io.imsave(colored, color.lab2rgb(img))
    io.imsave(gray, color.rgb2gray(color.lab2rgb(img)))

In [32]:
path = "input/"
image1 = "woman.jpg"
L, AB = preprocessImg(path+image1)

Ltrain = np.array(Ltrain, dtype=float)
ABtrain = np.array(ABtrain, dtype=float)
print(Ltrain.shape)
print(ABtrain.shape)
try:
    loaded_model = tf.keras.models.load_model('./tmp/model')
except:
    loaded_model = model(Ltrain, ABtrain)
# loaded_model.summary()

loaded_model = model(Ltrain, ABtrain)
# 

(288, 360, 360, 1)
(288, 360, 360, 2)
Epoch 1/10
29/29 [==============================] - 32s 1s/step - loss: 0.0297
Epoch 2/10
29/29 [==============================] - 32s 1s/step - loss: 0.0110
Epoch 3/10
29/29 [==============================] - 31s 1s/step - loss: 0.0104
Epoch 4/10
29/29 [==============================] - 32s 1s/step - loss: 0.0108
Epoch 5/10
29/29 [==============================] - 32s 1s/step - loss: 0.0103
Epoch 6/10
29/29 [==============================] - 31s 1s/step - loss: 0.0102
Epoch 7/10
29/29 [==============================] - 31s 1s/step - loss: 0.0100
Epoch 8/10
29/29 [==============================] - 31s 1s/step - loss: 0.0099
Epoch 9/10
29/29 [==============================] - 31s 1s/step - loss: 0.0097
Epoch 10/10
29/29 [==============================] - 32s 1s/step - loss: 0.0098


2022-05-22 18:25:45.985121: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./tmp/model/assets
Epoch 1/10
29/29 [==============================] - 32s 1s/step - loss: 0.1050
Epoch 2/10
29/29 [==============================] - 31s 1s/step - loss: 0.0108
Epoch 3/10
29/29 [==============================] - 31s 1s/step - loss: 0.0105
Epoch 4/10
29/29 [==============================] - 31s 1s/step - loss: 0.0104
Epoch 5/10
29/29 [==============================] - 31s 1s/step - loss: 0.0129
Epoch 6/10
29/29 [==============================] - 31s 1s/step - loss: 0.0101
Epoch 7/10
29/29 [==============================] - 31s 1s/step - loss: 0.0101
Epoch 8/10
29/29 [==============================] - 31s 1s/step - loss: 0.0100
Epoch 9/10
29/29 [==============================] - 31s 1s/step - loss: 0.0100
Epoch 10/10
29/29 [==============================] - 31s 1s/step - loss: 0.0098
INFO:tensorflow:Assets written to: ./tmp/model/assets


In [8]:
print(loaded_model.evaluate(Ltrain, ABtrain, batch_size = 10))

29/29 [==============================] - 7s 243ms/step - loss: 0.0098
0.00982951931655407


In [30]:
saveImg(convertImg('input/woman.jpg', loaded_model), "woman2")


(1, 360, 360, 1)
(1, 360, 360, 2)


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


In [38]:
for filename in os.listdir('test/'):
    if os. path. isdir('test/'+filename):
        continue
    try:
        saveImg(convertImg('test/'+filename, loaded_model), filename)
    except:
        continue

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0